## RSV Retry Min
Took out the data from 2020-2021, 2021-2022.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from statistics import mean
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from statistics import mean
import math

FUTURE_PREDICTION = 4

def findmax(x):
    maximum = 0
    for number in x:
        maximum = max(x)
    return maximum

def findmean(x):
    m = 0
    m = mean(x)
    return m

def findmin(x):
    m = 0
    m = min(x)
    return m

def CreateMean(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Mean'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            mean = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Mean'])
            mean = findmean(list)
            dataframe.at[x,f'{title}_Mean'] = mean
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMax(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Max'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            maximum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Max'])
            maximum = findmax(list)
            dataframe.at[x,f'{title}_Max'] = maximum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMin(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Min'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            minimum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Min'])
            minimum = findmin(list)
            dataframe.at[x,f'{title}_Min'] = minimum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe
    
def DataPreprocess(city):
    virus_df = pd.read_csv('Data/virus.csv')
    virus_df['Date'] = pd.to_datetime(virus_df['Date'])
    virus_df['year'] = virus_df['Date'].apply(lambda date: date.year)
    virus_df['month'] = virus_df['Date'].apply(lambda date: date.month)
    virus_df_new = virus_df[['Date','month','year',f'{city}']]
    virus_df_new.rename(columns = {f'{city}' : f'Virus_Count_{city}'}, inplace = True)

    temp_df = pd.read_csv(f'Data/{city}_temp.csv')
    pressure_df = pd.read_csv(f'Data/{city}_pressure.csv')
    hours_df = pd.read_csv(f'Data/{city}_hours.csv')
    rainfall_df = pd.read_csv(f'Data/{city}_rainfall.csv')

    temp_df['Date'] = pd.to_datetime(temp_df['Date'])
    temp_df['year'] = temp_df['Date'].apply(lambda date: date.year)
    temp_df['month'] = temp_df['Date'].apply(lambda date: date.month)

    pressure_df['Date'] = pd.to_datetime(pressure_df['Date'])
    pressure_df['year'] = pressure_df['Date'].apply(lambda date: date.year)
    pressure_df['month'] = pressure_df['Date'].apply(lambda date: date.month)
    pressure_df = pressure_df.sort_values(by="Date")
    pressure_df = pressure_df.reset_index(drop=True)

    hours_df['Date'] = pd.to_datetime(hours_df['Date'])
    hours_df['year'] = hours_df['Date'].apply(lambda date: date.year)
    hours_df['month'] = hours_df['Date'].apply(lambda date: date.month)
    hours_df = hours_df.sort_values(by="Date")
    hours_df = hours_df.reset_index(drop=True)

    rainfall_df['Date'] = pd.to_datetime(rainfall_df['Date'])
    rainfall_df['year'] = rainfall_df['Date'].apply(lambda date: date.year)
    rainfall_df['month'] = rainfall_df['Date'].apply(lambda date: date.month)
    rainfall_df = rainfall_df.sort_values(by="Date")
    rainfall_df = rainfall_df.reset_index(drop=True)

    list = []
    for x in temp_df.index:
        t = temp_df.iloc[x][f'{city}']
        svp = 6.1078*10**(7.5*t/(t+237.3))
        e = pressure_df.iloc[x][f'{city}']
        h = e/svp*100
        list.append(h)
        
    humidity_df = pd.DataFrame(data=list)
    humidity_df.rename(columns = {0:f'{city}'}, inplace = True)
    
    min_temp_df = CreateMin(temp_df, 'Temperature',f'{city}')
    min_pressure_df = CreateMin(pressure_df, 'Pressure',f'{city}')
    min_hours_df = CreateMin(hours_df, 'Hours',f'{city}')
    min_rainfall_df = CreateMin(rainfall_df, 'Rainfall',f'{city}')
    min_humidity_df = CreateMin(humidity_df, 'Humidity',f'{city}')

    df = pd.concat([virus_df_new,min_temp_df, min_pressure_df,min_hours_df,min_rainfall_df,min_humidity_df], axis=1)
    
    df = df.loc[:,~df.columns.duplicated()]
    
    df['VirusCRM'] = df['Virus_Count_Tokyo'].rolling(14, center = True).mean()

    for i in df.index:
        if i == 0:
            df.at[i,'Gradient'] = 1
        else:
            if df.iloc[i-1]['VirusCRM'] == 0:
                if df.at[i,'VirusCDM'] > 0:
                    df.at[i,'Gradient'] = 1
                else:
                    df.at[i,'Gradient'] = 0
            else:
                grd = df.iloc[i]['VirusCRM']/(df.iloc[i-1]['VirusCRM'])
                df.at[i,'Gradient'] = grd
    
    for i in df.index:
        if (df.iloc[i]['Gradient'] >= 1.1) and (df.iloc[i+1]['Gradient'] >= 1.1) and (df.iloc[i+2]['Gradient'] >= 1.1):
            df.at[i,'Spreading'] = 1
        else:
            df.at[i,'Spreading'] = 0
    df = df.astype({'Spreading':'int8'})

    df_virus_count = df[f'Virus_Count_{city}']
    df.drop(columns=['VirusCRM',f'Virus_Count_{city}','year','Gradient'], axis=1, inplace=True)
    df_labels = df[['Date']]
    FUTURE_PREDICTION = 4
    for i in range(FUTURE_PREDICTION):
        df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)
    temp = df[["Date", "Spreading"]]
    for i in range(FUTURE_PREDICTION):
        target_column = f"T+{(i+1)*7}"
        df_labels = df_labels.merge(temp, 
                                how ="left",
                                left_on = (target_column),
                                right_on=("Date"),
                                suffixes = ("","_y")
        )
        df_labels.rename(columns={"Spreading":f"{target_column}_Spreading"},inplace=True)
    df_labels.drop(columns=["Date_y"],inplace=True)

    for i in range(FUTURE_PREDICTION):
        df_labels.drop(columns =[f"T+{(i+1)*7}"],inplace=True,axis=1 )

    df = df.merge(df_labels,
             how='left',
             on = ["Date"])
    df.dropna(inplace=True)

    X = df.drop(columns=['T+7_Spreading','T+14_Spreading','T+21_Spreading','T+28_Spreading'],axis=1)
    y1 = df['T+21_Spreading']
    y2 = df['T+7_Spreading']
    y3 = df['T+14_Spreading']
    y4 = df['T+28_Spreading']

    return X, y1, y2, y3, y4, df, df_virus_count

In [3]:
X, y1, y2, y3, y4, df, df_virus_count = DataPreprocess('Tokyo')

/Users/miyanishio/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/k5/3yc13bs57f7637937yjlstth0000gn/T/ipykernel_11223/2190827270.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)


In [4]:
X

,Date,month,Temperature_Min,Pressure_Min,Hours_Min,Rainfall_Min,Humidity_Min,Spreading
0,2012-09-10,9,27.3,25.2,3.5,0.0,67.299614,0
1,2012-09-17,9,27.9,22.8,4.6,0.0,59.974362,0
2,2012-09-24,9,20.3,20.1,0.0,0.0,70.685242,0
3,2012-10-01,10,20.4,15.4,0.0,0.0,61.171464,0
4,2012-10-08,10,19.1,14.9,0.0,0.0,64.521610,0
...,...,...,...,...,...,...,...,...
477,2021-11-01,11,13.6,9.1,0.0,0.0,52.345705,0
478,2021-11-08,11,14.9,9.9,4.6,0.0,56.221277,0
479,2021-11-15,11,13.1,8.2,0.0,0.0,49.152989,0
480,2021-11-22,11,12.5,9.1,0.0,0.0,58.809630,0


In [5]:
X = X.drop(columns=['Date'])

In [6]:
X = X[0:400] #until 2020 05-04
y1 = y1[0:400]
y2 = y2[0:400]
y3 = y2[0:400]
y4 = y2[0:400]

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler1 = StandardScaler()
X_train, X_test = X[0:280], X[280:400]
y1_train, y1_test = y1[0:280], y1[280:400]

In [8]:
scaled_X_train = scaler1.fit_transform(X_train)
scaled_X_test = scaler1.transform(X_test)

In [9]:
from sklearn.svm import SVC

In [10]:
param_grid = {
 'C': [0.1,1, 10, 100, 1000], 
 'gamma': [1,0.1,0.01,0.001], 
 'kernel': ['rbf','linear','sigmoid']
} 

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
grid = GridSearchCV(SVC(),param_grid,verbose=3)

In [13]:
grid.fit(scaled_X_train,y1_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.821 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.821 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.929 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.839 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=1, kernel=sigmoid

[CV 3/5] END .........C=10, gamma=1, kernel=rbf;, score=0.911 total time=   0.0s
[CV 4/5] END .........C=10, gamma=1, kernel=rbf;, score=0.893 total time=   0.0s
[CV 5/5] END .........C=10, gamma=1, kernel=rbf;, score=0.875 total time=   0.0s
[CV 1/5] END ......C=10, gamma=1, kernel=linear;, score=0.821 total time=   0.0s
[CV 2/5] END ......C=10, gamma=1, kernel=linear;, score=0.929 total time=   0.0s
[CV 3/5] END ......C=10, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 4/5] END ......C=10, gamma=1, kernel=linear;, score=0.893 total time=   0.0s
[CV 5/5] END ......C=10, gamma=1, kernel=linear;, score=0.804 total time=   0.0s
[CV 1/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.857 total time=   0.0s
[CV 2/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.839 total time=   0.0s
[CV 3/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.804 total time=   0.0s
[CV 4/5] END .....C=10, gamma=1, kernel=sigmoid;, score=0.750 total time=   0.0s
[CV 5/5] END .....C=10, gamm

[CV 2/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.929 total time=   0.0s
[CV 3/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.893 total time=   0.0s
[CV 4/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.893 total time=   0.0s
[CV 5/5] END C=100, gamma=0.001, kernel=sigmoid;, score=0.893 total time=   0.0s
[CV 1/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.839 total time=   0.0s
[CV 2/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.857 total time=   0.0s
[CV 3/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.821 total time=   0.0s
[CV 4/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.875 total time=   0.0s
[CV 5/5] END .......C=1000, gamma=1, kernel=rbf;, score=0.857 total time=   0.0s
[CV 1/5] END ....C=1000, gamma=1, kernel=linear;, score=0.821 total time=   3.8s
[CV 2/5] END ....C=1000, gamma=1, kernel=linear;, score=0.929 total time=   2.6s
[CV 3/5] END ....C=1000, gamma=1, kernel=linear;, score=0.893 total time=   2.2s
[CV 4/5] END ....C=1000, gam

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'sigmoid']},
             verbose=3)

In [14]:
grid.best_params_

{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}

In [15]:
model_SVC = SVC(C=0.1, gamma=1,kernel='linear')

In [16]:
model_SVC.fit(scaled_X_train, y1_train)

SVC(C=0.1, gamma=1, kernel='linear')

In [17]:
preds = model_SVC.predict(scaled_X_test)

In [18]:
from sklearn.metrics import classification_report,confusion_matrix

In [19]:
print(classification_report(y1_test,preds))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       108
         1.0       0.50      0.50      0.50        12

    accuracy                           0.90       120
   macro avg       0.72      0.72      0.72       120
weighted avg       0.90      0.90      0.90       120

